# Grama: Fitting Functions

*Purpose*: Often, we will be able to write down a function (say, using physical laws) that maps input physical quantities to some output of interest. However, we may not have the means to *set values* for those physical inputs. Or, we might be using the model to *infer* values for some physical inputs (i.e. a measurement model). In this case, we can use statistical tools to *fit* the parameters in a function using a dataset. This exercise will introduce these ideas with a particular case study.


## Setup


In [ ]:
import grama as gr
DF = gr.Intention()
%matplotlib inline

# Fitting

Recall that there are four classes of verb in grama; *fitting* verbs take data as an input and produce a model as an output. There are various ways to fit a model; in this exercise we'll focus on fitting a *parameterized* model using  *nonlinear least squares*.



![Grama verb class diagram](./images/verb-classes.png)


## Trajectory Data

We're going to use the following trajectory dataset to fit a function predicting the trajectory. Our goal in this case study is as follows:

> Our goal is to fit a trajectory model and accurately predict the range of the projectile.

Note that our data only covers the beginning of the trajectory; we can't see where it lands (hence, we don't know its range).


In [ ]:
# NOTE: No need to edit
from grama.data import df_trajectory_windowed
(
    df_trajectory_windowed
    >> gr.ggplot(gr.aes("x", "y"))
    + gr.geom_point()
)

# EMA of Proposed Model

Before we jump immediately to using statistical procedures to fit a function, let's first use EMA to understand the model we're going to fit.


## Trajectory model

The following code loads a model for the trajectory of a projectile subject to a *linear drag force*.


In [ ]:
from grama.models import make_trajectory_linear
md_trajectory = make_trajectory_linear()
md_trajectory

### Linear drag model

The drag $\vec{F}_d$ acting on the projectile in this model has a magnitude equal to

$$F_d \propto - m \tau s$$

where $m$ is the mass of the projectile, $s$ is its speed, and $\tau$ is the *time constant* associated with the drag law.

We can make sense of the model by inspecting the equations in the model. However, a complementary way to gain understanding is to do a little *exploratory model analysis*. You'll use EMA below to understand the model's behavior.


## Sinews and bounds

One "gotcha" with `ev_sinews()` is that it can only sweep inputs with *finite bounds*. For variables with one-sided bounds, such as `[0.1, inf]`, we cannot sweep values all the way to infinity! Instead, grama will set the inputs with one-sided bounds to their lower bound, and sweep the remaining variables. We need to set finite bounds in order to use `gr.ev_sinews()`.


### __q1__ Sweep values of `tau`

Update the model to enable a sweep over values of `tau`. How does changing `tau` affect the trajectory? Answer the questions under *observations* below.

*Hint*: Remember that `gr.cp_bounds()` allows you to adjust the bounds of a grama model. Note that you may need to adjust the bounds of time `t` as well as `tau` to make an informative plot.


In [ ]:
(
    md_trajectory

    >> gr.ev_sinews(df_det="swp", n_sweeps=4)
    >> gr.pt_auto()
)

*Observations*

- Once you get a sweep of `tau` displayed, what do each of the curves in the `tau` column represent?
  - (Your response here)
- What affect does `tau` have on the trajectory range `x`?
  - (Your response here)
- What affect does `tau` have on the trajectory height `y`?
  - (Your response here)


### __q2__ Plot a few trajectories

Complete the code below to sweep over values of `tau` and `t`. Compare these results with what you found above. Answer the questions under *observations* below.


In [ ]:
# TASK: Sweep tau and t to create trajectories
(
    md_trajectory
    >> gr.ev_df(
        df=gr.df_grid(
            u0=0.1,
            v0=0.1,
            # TASK: Complete the code to sweep tau and t

        )
    )
    >> gr.ggplot(gr.aes("x", "y", color="factor(tau)"))
    + gr.geom_line()
)

*Observations*

- What affect does `tau` have on the trajectory?
  - (Your response here)
- Imagine that `y == 0` represents the ground. Are the endpoints of these trajectories physically reasonable?
  - (Your response here)


# Fitting with Least Squares

Above, we used EMA to make sense of the model's behavior, but we only guessed at parameter values. Now let's use a statistical fitting procedure to get reasonable values for the inputs.


## The `ft_nls()` routine

The `gr.ft_nls()` routine is a *fitting* routine; it takes in a dataset and returns a (fitted) model. NLS stands for [*nonlinear least squares*](https://en.wikipedia.org/wiki/Non-linear_least_squares), a general-purpose way to fit a parameterized function to a dataset. There is a great deal of statistical theory underpinning NLS, which we will not cover in this exercise. The short version is: NLS seeks input (parameter) values of a function that give the best "agreement" between the function's output and measured output values.

There are a number of practical concerns to using NLS effectively, which we'll study below.


### __q3__ Run `ft_nls()`

Use `gr.ft_nls()` to fit the model `md_trajectory` to the data `df_trajectory_windowed`. Answer the questions under *observations* below.

*Hint*: Make sure to read the documentation of an unfamiliar function to learn how to use it!


In [ ]:
# TASK: Fit a model to the data
md_fit = (
    df_trajectory_windowed
    ## TODO: Fit the model md_trajectory

)

## NOTE: Use this to check your work
assert \
    isinstance(md_fit, gr.Model), \
    "md_fit is not a model; make sure to fit a model"

md_fit

*Observations*

- The `gr.ft_nls()` routine should print some diagnostic information; was the optimizer `success`ful in finding best-fit input values? How do you know?
  - (Your response here)
- The `gr.ft_nls()` starts from an "initial guess" for each of the inputs; these are shown in the diagnostic information as the input names with the suffix `_0`. What were the initial guess values for the inputs?
  - (Your response here)


The fitting diagnostics only tell us so much; it is a far better idea to compare predictions from the model directly against the original data.


### __q4__ Assess the fit

Evaluate the fitted model `md_fit` to compare the predicted values against the original dataset. Answer the questions under *observations* below.


In [ ]:
# TASK: Compare the model predictions with
# the original data df_trajectory_windowed
(
    md_fit
    ## TODO: Evaluate the model at df_trajectory_windowed

    
    >> gr.ggplot(gr.aes("x", "y"))
    + gr.geom_line()
    + gr.geom_point(data=df_trajectory_windowed)
)

*Observations*

- How well does the fitted model (solid line) agree with the measured values (dots)?
  - (Your response here)


## Rough estimation

Clearly, using completely-arbitrary guesses for the initial parameter values has led to a terrible fit. Let's do some back-of-the envelope calculations to get a rough sense of reasonable parameter values.

The horizontal velocity is the time-derivative of horizontal position; we can approximate this with a simple finite-difference

$$u = \frac{dx}{dt} \approx \frac{u(t_2) - u(t_1)}{t_2 - t_1}$$

The `gr.lead()` function allows us to access values in the following row; we can use this to implement differences like $u(t_2) - u(t_1)$. In code, this would be `gr.lead(DF.u) - DF.u`.

We can estimate the velocity of the projectile directly using the position and time data, but we'll have to work a little harder to estimate a reasonable value for `tau`. To do this, let's turn to the drag law: 

$$F_d \propto - m \tau s$$

Re-arranging to isolate $\tau$, we find

$$F_d/m/s \propto - \tau$$

or

$$|a_d / s| \propto |\tau|$$

where $a_d$ is the acceleration due to drag. Note that this is *not* the total acceleration! This is just the acceleration due to drag, which we can't access from the data alone. However, we can use the *total* acceleration to set a reasonable first-guess for the value of `tau`.

The following code implements the calculations described above and computes summary statistics over the estimated `u, v, tau` values.


In [ ]:
# NOTE: No need to edit
(
    df_trajectory_windowed
    # Estimate velocity components
    >> gr.tf_mutate(
        u=(gr.lead(DF.x) - DF.x) / (gr.lead(DF.t) - DF.t),
        v=(gr.lead(DF.y) - DF.y) / (gr.lead(DF.t) - DF.t),
    )
    # Compute speed
    >> gr.tf_mutate(s=gr.sqrt(DF.u**2 + DF.v**2))
    # Estimate acceleration
    >> gr.tf_mutate(
        a=(gr.lead(DF.s) - DF.s) / (gr.lead(DF.t) - DF.t)
    )
    # Estimate drag time constant
    >> gr.tf_mutate(tau=gr.abs(DF.a / DF.s))
    
    >> gr.tf_select("u", "v", "tau")
    >> gr.tf_describe()
)

This *does not* directly provide the "best" parameter values for `u, v, tau`. However, this gets us in the right *ballpark*. Choosing values similar to the above will be much more effective than picking completely-arbitrary initial guess values.


### __q5__ Choose reasonable initial parameters

Override the default initial parameter guess of `ft_nls()` by adding a keyword argument. Using the rough estimates above, you should be able to achieve a fit of the data that is quite reasonable. Answer the questions under *observations* below.

*Hint*: Remember to consult the documentation for `ft_nls()` to see how to use its arguments!


In [ ]:
# TASK: Set a reasonable initial guess for the input values
md_fit_init = (
    df_trajectory_windowed
    >> gr.ft_nls(
        md=md_trajectory,

)

## NOTE: Use this to check your work
(
    md_fit_init
    >> gr.ev_df(df_trajectory_windowed)
    
    >> gr.ggplot(gr.aes("x", "y"))
    + gr.geom_line()
    + gr.geom_point(data=df_trajectory_windowed)
)

*Observations*

- How well does the trend fit the data? Is the fit *perfect*?
  - (Your response here)
- How does your initial guess for `tau` compare with the fitted value? What might account for this?
  - (Your response here)
- How does your initial guess for `u0` compare with the fitted value? What might account for this?
  - (Your response here)


# Model Assessment

We have successfully fit a function to data! However, before we start using this model to make predictions, we should carry out some *model assessment* studies to check that this is a reasonable model. The next few tasks will guide you through some useful assessment techniques.


## Quantifying uncertainty

The `gr.ft_nls()` routine allows us to quantify the uncertainty in the fitted values by fitting a joint distribution for the inputs. As we saw above, the fit of the function to the data is *not* perfect; therefore, the noise in the measured values implies we cannot estimate the input values perfectly. Setting `uq_method="linpool"` in `gr.ft_nls()` accounts for this by finding a "best-fit" value, but additionally fitting a normal distribution to account for the noise in the outputs.


In [ ]:
# NOTE: No need to edit
md_fit_uq = (
    df_trajectory_windowed
    >> gr.ft_nls(
        md=md_trajectory,
        df_init=gr.df_make(
            u0=18,
            v0=18,
            tau=16,
        ),
        ## NOTE: This fits a distribution for the inputs
        uq_method="linpool",
    )
)
md_fit_uq 

Since this is a grama model, we can use the same kind of sampling tools we've seen before to inspect the uncertainty in the fitted input values.


### __q6__ Inspect plausible input values

The following code visualizes the input uncertainty of the fitted values. Run the code below, and answer the questions under *observations* below.


In [ ]:
# TASK: Run and inspect
(
    md_fit_uq
    >> gr.ev_sample(n=1e3, df_det="nom", skip=True)
    >> gr.pt_auto()
)

*Observations*

- How variable are each of the inputs?
  - (Your response here)


### __q7__ Inspect plausible trajectories

Draw a random sample of trajectories to study the uncertainty in in the outputs. Make sure to pick a reasonable range of time values `t` so that you can see both the beginning and end (ground-strike at `y==0`) of the trajectory. Answer the questions under *observations* below.


In [ ]:
# TASK: Evaluate the model
(
    md_fit_uq
    ## TODO: Draw a random sample of trajectories

    
    ## NOTE: No need to edit below here; this will visualize your results
    >> gr.ggplot(gr.aes("x", "y"))
    # NOTE: The `group` aesthetic allows us to draw
    # individual lines for each value of u0; otherwise
    # the lines would all be connected
    + gr.geom_line(gr.aes(group="u0"), alpha=1/5, color="grey")
    # Add the data
    + gr.geom_point(data=df_trajectory_windowed)
    # Clean up the visual
    + gr.theme_minimal()
    + gr.labs(
        x="Horizontal Position (m)",
        y="Vertical Position (m)",
    )
)

*Observations*

- Compare the ensemble of trajectories (grey lines) against the measured values (black dots); is the uncertainty in the trajectory comparable to the observed variability in the measured values? How do you know?
  - (Your response here)
- According to the fitted model, what is a plausible final range for the projectile? (In what band of `x` values does it hit the ground at `y == 0`?)
  - (Your response here)


## Model Validation

One of the *most important* assessments we should carry out is *model validation*. When validating a model, we use a dataset that was *not* used to fit the model in order to check how well the model agrees with physical reality. This is to help avoid the statistical phenomenon of [overfitting](https://en.wikipedia.org/wiki/Overfitting).


## Validation data

Let's load a dataset that is appropriate for validation; the following data is from the same trajectory as `df_trajectory_windowed`, but it consists of independent observations, and it runs for a longer span of time values.


In [ ]:
## NOTE: No need to edit
from grama.data import df_trajectory_full
(
    df_trajectory_full
    >> gr.tf_mutate(source="Full")
    >> gr.tf_bind_rows(
        df_trajectory_windowed
        >> gr.tf_mutate(source="Windowed")
    )
    >> gr.ggplot(gr.aes("x", "y", color="source"))
    + gr.geom_point()
)

### __q8__ Compare the fit to validation data

Compare the predictions from the fitted model `md_fit_uq` to compare an ensemble of trajectories against the validation data `df_trajectory_full`. Answer the questions under *observations* below.

*Hint*: This part has no starter code! Use what you learned above to carry out this comparison.


In [ ]:
# TASK: Compare the fitted model to the validation data;
# make sure to quantify the parameter uncertainty
(
    md_fit_uq
 
)

*Observations*

- How well does the model fit the data? Does the model perform better in some regions than others?
  - (Your response here)


## The dramatic reveal...

It turns out the model does *not* accurately predict the trajectory's range! This is because our model assumes a *linear* drag law $F_d \propto - m \tau s$, while in reality the data were generated using a *quadratic* drag law $F_d \propto - m b s^2$. This qualitative difference between the model and the data-generating process leads to a mismatch between predicted and measured values that we can't overcome without changing the model's *form*.

This is why validation is so important: It is difficult to discover this kind of *model form error* without a validation study. In practice, data scientists will use a [*train-validation split*](https://en.wikipedia.org/wiki/Training,_validation,_and_test_data_sets) to help construct this sort of validation study.


# Further Reading

- This exercise is [based on a chapter](https://zdelrosario.github.io/uq-book-preview/02_diagnosing/model-form-error.html) from a forthcoming book I'm writing. You can see that draft chapter for more details on this case study.
- My most-highly recommended book for a deeper treatment of the train-validation split (and related ideas) is "An Introduction to Statistical Learning," which is [freely available online](https://www.statlearning.com/).
